In [44]:
import tqdm
import numpy as np
import pandas as pd
import plotly.express as px
from typing import List



In [34]:

class Miner:
    def __init__(self, mean, std, fail_rate):
        self.mean = mean
        self.std = std
        self.fail_rate = fail_rate
    
    def reward(self):
        if np.random.rand() < self.fail_rate:
            return 0
        else:
            return np.random.normal(self.mean, self.std)

num_miners = 1000
miners = [Miner(mean=np.random.randn()*0.04+0.3, std=np.random.randn()*0.008+0.05, fail_rate=np.random.rand()*0.1) for _ in range(1000)]


In [35]:
df = pd.DataFrame([{'i':i, 'mean':miners[i].mean, 'std':miners[i].std, 'fail_rate':miners[i].fail_rate, 'reward_mean':miners[i].mean * (1-miners[i].fail_rate)} for i in range(num_miners)])
px.scatter(df, x='mean', y='std', color='reward_mean',
           title='Distribution of Miner Reward Means and Stds',
           opacity=0.5,
           width=800, height=600, template='plotly_dark')

In [50]:
def apply_reward_func(raw_rewards, p):
    exponent = (p**6.64385619)*100 # 6.64385619 = ln(100)/ln(2) -> this way if p=0.5, the exponent is exatly 1
    positive_rewards = np.clip(raw_rewards, 0, np.inf)
    normalised_rewards = positive_rewards / np.max(positive_rewards)
    post_func_rewards = normalised_rewards ** exponent
    all_rewards = post_func_rewards
    all_rewards[raw_rewards < 0] = 0
    return all_rewards

def get_raw_rewards(miners : List[Miner]):
    return np.array([miner.reward() for miner in miners])

def get_rewards(miners : List[Miner], p):
    raw_rewards = get_raw_rewards(miners)
    rewards = apply_reward_func(raw_rewards, p)
    return {'raw_rewards':raw_rewards, 'rewards':rewards}

def run_simulation(num_miners=1000, p=0.5, num_steps=1000, ema_alpha=0.1, sample_size=100):
    miners = [Miner(mean=np.random.randn()*0.04+0.3, std=np.random.randn()*0.008+0.05, fail_rate=np.random.rand()*0.1) for _ in range(1000)]

    ema_scores = np.zeros(len(miners))
    ema_history = []

    for i in range(num_steps):
        indices = np.random.choice(num_miners, sample_size, replace=False)
        selected_miners = [miners[i] for i in indices]
        rewards = get_rewards(selected_miners, p)
        ema_scores[indices] = ema_scores[indices] * (1 - ema_alpha) + rewards['rewards'] * ema_alpha
        ema_history.append(ema_scores)


    df = pd.DataFrame([{'i':i, 'mean':miners[i].mean, 'std':miners[i].std, 'fail_rate':miners[i].fail_rate, 'reward_mean':miners[i].mean * (1-miners[i].fail_rate)} for i in range(num_miners)])
    df['ema_score'] = ema_scores
    return df

df = run_simulation(num_miners=1000, p=0.5, num_steps=1000, ema_alpha=0.1, sample_size=100)
px.scatter(df, x='mean', y='ema_score', color='fail_rate', 
           labels={'mean':'Mean Reward', 'ema_score':'EMA Score', 'fail_rate':'Fail Rate'},
           opacity=0.5, color_continuous_scale='BlueRed',
           title='Relationship between Miner Performance and EMA Score',
           template='plotly_dark', width=800, height=600)

In [42]:
# Spearman correlation between mean and ema_score
spearman_corr = df[['mean', 'ema_score']].corr(method='spearman')
print(f'Perfect preservation of miner ranking is defined as the Spearman correlation of 1.0')
print(f'Spearman correlation between mean and ema_score: {spearman_corr.iloc[0,1]:.4f}')



Perfect preservation of miner ranking is defined as the Spearman correlation of 1.0
Spearman correlation between mean and ema_score: 0.8659


In [48]:
results = []
for i in tqdm.tqdm(range(1000)):
    p = np.random.rand()
    sample_size = np.random.randint(10, 1000)
    # NOTE: assumes that 100 steps is enough to converge to the ema score
    df = run_simulation(num_miners=1000, p=p, num_steps=100, ema_alpha=0.1, sample_size=sample_size)
    spearman_corr = df[['mean', 'ema_score']].corr(method='spearman')
    
    results.append({'i':i, 'p':p, 'spearman_corr':spearman_corr.iloc[0,1], 'sample_size':sample_size})
    # print(f'Spearman correlation between mean and ema_score: {spearman_corr.iloc[0,1]:.4f}')
    
df_results = pd.DataFrame(results)
df_results

100%|██████████| 1000/1000 [00:59<00:00, 16.70it/s]


,i,p,spearman_corr,sample_size
0,0,0.857295,0.810326,471
1,1,0.525611,0.165320,11
2,2,0.630223,0.929354,820
3,3,0.218230,0.101264,15
4,4,0.252963,0.011626,445
...,...,...,...,...
995,995,0.239370,0.002060,349
996,996,0.028879,-0.019265,82
997,997,0.939896,0.790059,714
998,998,0.410998,0.510370,697


In [49]:
px.scatter(df_results, x='p', y='spearman_corr', color='sample_size',
           labels={'p':'p', 'spearman_corr':'Spearman Correlation', 'sample_size':'Sample Size'},
           opacity=0.5, color_continuous_scale='BlueRed',
           title='Relationship between p and Spearman Correlation',
           template='plotly_dark', width=800, height=600)